# Model voor backtesten maken

Op 2020 data, genormaliseerd

Script komt grotendeels overeen met script 2d


In [26]:
# importeren packages
import pandas as pd
import numpy as np
#conda install -c anaconda py-xgboost #installeer XGBoost
import xgboost
#from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler 
import pickle
from matplotlib import pyplot
#import seaborn as sns #visualisation
#import matplotlib.pyplot as plt #visualisation

# setten source path: later moet alle brondata naar datalake
source_path ="P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\"
# alle tussentijdse data voorlopig in dezelfde map op een schijf opslaan in
save_path = "P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\"


#%matplotlib inline

In [2]:
# display meer regels
pd.set_option('display.max_rows', 500)

### Inladen data

In [3]:
# inladen bestand met scores als sc
sc = pd.read_excel(source_path + 'buurten voor experts 2e ronde.xlsx', sheet_name='buurten')
# alleen selecteren kolommen voor scores en oordelen
sc = sc[['buurtcode','EE','TK']]

# inladen bestand met alle buurten en onafhankelijke variabelen als all
buurtendata = pd.read_csv(source_path + 'buurten.csv', encoding='utf-16')

C:\Users\rondea\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# merge op buurtcode (die van 2019)
df = pd.merge(sc,buurtendata, on='buurtcode')

### Maak splits in test en training set

In [5]:
# hernoem RLBBEV naar zonder jaartal, zodat we kunnen vergelijken tussen 2016 en 2018
df.rename(columns={"RLBBEV18":"RLBBEV"}, inplace=True) 

In [6]:
# alle kolommen! behalve enkele omschhrijvingen, onnodige wijkcodes, marktpotentie want String 
# feature_cols = ['woxdelta5jrGem', 'AantalCafe3', 'woxdelta5jr', 'ar60', 'wox',
#        'woxdelta1jrGem', 'AantalBios5', 'woxm2WG']



# minus de kolommen die niet in de 2016 dataset aanwezig zijn
feature_cols = ['AantalCafe3', 'woxdelta5jr', 'woxm2', 'woxww',
       'woxdelta5jrGem', 'wox', 'RLBBEV',
       'RLBVRZ16', 'AantalLevensm3', 'woxm2WG', 'aand1945-1970',
       'woxdelta5jrWGabs', 'woxdelta1jrGem', 'AantalPodiumk10',
       'wv2000-2010', 'AantalBios5']




#zonder wox
#feature_cols = ['buurtcode','inw', 'bevdh', 'oadh', 'hh', 'inw65', 'KL16', 'KL18', 'VKL1618',
#                'RLBRMTR18', 'RLBWON18', 'RLBBEV18', 'RLBVRZ18', 'RLBVEI18', 'RLBFYS18', 
#                'AfstandSupermarkt', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3','AantalRest3', 'AantalKinderopvang1', 
#                'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5', 
#                'hh19inkLaag', 'hh30inkLaag', 'hh19inkMidden', 'hh30inkMidden', 'hh19inkHoog', 'hh30inkHoog', 
#                'aandeelMH19', 'auto30', 'auto60', 'ov30', 'ov60', 'fiets', 'banen'] 



#X = df[feature_cols]
y = df[['TK']]

In [7]:
# Normaliseren data
# Creating a scaled feature set
scaler = RobustScaler()
X = scaler.fit_transform(df[feature_cols])
X = pd.DataFrame(X)
X.columns = feature_cols

## hyperparameter search

- 3*10-fold cross-validation met random seeds
- en holdout set om te verzekeren dat je niet je cross-validatie overfit

- gekozen om gewoon bomen te gebruiken als booster
- model in principe niet exact reproducible ook als we random state
- base_score zou lager kunnen omdat we voor 3 groepen classificeren, maar we geven hem genoeg stappen om te trainen dus deze maakt niet echt uit




In [9]:
# Laad model
clf = xgboost.XGBClassifier(seed=2)

In [30]:
clf.__dict__

{'max_depth': 3,
 'learning_rate': 0.1,
 'n_estimators': 100,
 'verbosity': 1,
 'silent': None,
 'objective': 'binary:logistic',
 'booster': 'gbtree',
 'gamma': 0,
 'min_child_weight': 1,
 'max_delta_step': 0,
 'subsample': 1,
 'colsample_bytree': 1,
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'base_score': 0.5,
 'missing': nan,
 'kwargs': {},
 '_Booster': None,
 'seed': 2,
 'random_state': 0,
 'nthread': None,
 'n_jobs': 1,
 'importance_type': 'gain'}

In [151]:
# test hyperparameter search input (per parameter terugbrengen naar minder waarden)
#    "max_depth"        : [2, 3, 4, 5, 6, 7, 9, 12, 19, 25],
#    "learning_rate"    : [0.01, 0.05, 0.10, 0.15, 0.20, 0.30, 0.50, 0.80],
#    "subsample"        : [0.1, 0.4, 0.6, 0.8, 1],
#    "min_child_weight" : [1, 3, 5, 7],
#    "gamma"            : [0.0, 0.1, 0.2, 0.3, 0.4],
#    "colsample_bytree" : [0.3, 0.4, 0.5, 0.7, 1],
#    "n_estimators"     : [10, 100, 250, 500]



# hyperparameter search input
params = {
    "max_depth"        : [2, 5, 7, 9],
    "learning_rate"    : [0.10, 0.15, 0.20],
    "subsample"        : [0.8, 1],
    "min_child_weight" : [3, 5, 7],
    "gamma"            : [0.1, 0.2, 0.3],
    "colsample_bytree" : [0.4, 0.5, 0.7],
    "n_estimators"     : [100, 250]
}

In [152]:
grid_search = GridSearchCV(
    estimator = clf,
    param_grid = params,
    scoring = 'f1_macro',
    n_jobs = 10,
    cv = 10,
    verbose = True
)

In [113]:
grid_search.fit(X, y)

Fitting 10 folds for each of 1296 candidates, totalling 12960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    9.2s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:   29.6s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:  1.2min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  2.0min
[Parallel(n_jobs=10)]: Done 1230 tasks      | elapsed:  3.0min
[Parallel(n_jobs=10)]: Done 1780 tasks      | elapsed:  4.3min
[Parallel(n_jobs=10)]: Done 2430 tasks      | elapsed:  5.9min
[Parallel(n_jobs=10)]: Done 3180 tasks      | elapsed:  7.7min
[Parallel(n_jobs=10)]: Done 4030 tasks      | elapsed:  9.8min
[Parallel(n_jobs=10)]: Done 4980 tasks      | elapsed: 12.4min
[Parallel(n_jobs=10)]: Done 6030 tasks      | elapsed: 15.1min
[Parallel(n_jobs=10)]: Done 7180 tasks      | elapsed: 18.3min
[Parallel(n_jobs=10)]: Done 8430 tasks      | elapsed: 21.8min
[Parallel(n_jobs=10)]: Done 9780 tasks      | elapsed: 26.1min
[Parallel(n_jobs=10)]: Done 11230 tasks      

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_...
                                     scale_pos_weight=1, seed=2, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=10,
             param_grid={'colsample_bytree': [0.4, 0.5, 0.7],
                         'gamma': [0.1, 0.2, 0.3],
                         'learning_rate': [0.1, 0.15, 0.2],
    

In [115]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

In [10]:
# splitten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

# voor geselecteerde kolommen, fase 2, TK

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

In [29]:
# maak object met geoptimaliseerde hyperparameters
model_2019 = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
model_2019.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

In [34]:
# sla model op
file_name = "model_backtesting.pkl"
pickle.dump(model_2019, open(file_name, "wb"))


In [15]:
y_pred = model_2019.predict(X_test)
y_train_pred = model_2019.predict(X_train)

In [16]:
train_accuracy = accuracy_score(y_train,y_train_pred)
test_accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test, y_pred)

In [17]:
train_accuracy

1.0

In [18]:
test_accuracy

0.9176470588235294

In [19]:
cm

array([[38,  1,  1],
       [ 1, 12,  3],
       [ 1,  0, 28]], dtype=int64)

In [21]:
model_2019.feature_importances_

array([0.09708524, 0.06382816, 0.0703288 , 0.05850822, 0.10547711,
       0.12422597, 0.0373393 , 0.04409109, 0.03296622, 0.06023987,
       0.03562468, 0.0368512 , 0.07125703, 0.05782312, 0.03101709,
       0.07333696], dtype=float32)

In [24]:
# maak dataframe met feature importances
fi = pd.DataFrame(model_2019.feature_importances_,X_train.columns)
fi.columns = ['feature_score']
fi.sort_values('feature_score', ascending=False, inplace=True)
fi['feature'] = fi.index
fi

,feature_score,feature
wox,0.124226,wox
woxdelta5jrGem,0.105477,woxdelta5jrGem
AantalCafe3,0.097085,AantalCafe3
AantalBios5,0.073337,AantalBios5
woxdelta1jrGem,0.071257,woxdelta1jrGem
woxm2,0.070329,woxm2
woxdelta5jr,0.063828,woxdelta5jr
woxm2WG,0.060240,woxm2WG
woxww,0.058508,woxww
AantalPodiumk10,0.057823,AantalPodiumk10


In [23]:
len(feature_cols)

16

In [ ]:
fi[fi['feature_score']>0.05]['feature'].values

In [35]:
#fi.to_csv('fi EE zonder wox.csv', encoding='utf-16', index=False)

### Predicten rest van buurten en maken bestand

In [61]:
prb = buurtendata[feature_cols]

In [62]:
pred = model.predict(prb)
pred = pd.DataFrame(pred,columns=['pred_TK'])
prb['pred_TK'] = pred

C:\Users\rondea\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [63]:
prb

,buurtcode,inw,bevdh,oadh,hh,inw65,KL16,KL18,VKL1618,RLBRMTR18,...,woxGem,woxdelta1jrGem,woxdelta5jrGem,woxwwGem,woxm2Gem,woxdelta5jrWGabs,woxdelta5jrWGrel,woxm2WG,banen,pred_TK
0,30000,2370,2812.0,1210.0,1315,700,6.0,6.0,4.0,-0.051487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41539.333333,c
1,30001,3035,1922.0,903.0,1335,640,6.0,6.0,4.0,-0.045828,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41444.708333,c
2,30002,5725,2012.0,1140.0,2665,1480,6.0,6.0,4.0,-0.185803,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39794.395833,c
3,30007,325,60.0,359.0,115,30,8.0,8.0,4.0,0.141117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42369.166667,c
4,30008,100,18.0,79.0,40,15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50376.500000,c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13619,19781901,110,2043.0,50.0,40,10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29348.666667,c
13620,19782001,10,2.0,18.0,5,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242544.333333,c
13621,19782002,440,3870.0,75.0,180,95,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242544.333333,c
13622,19782003,150,536.0,36.0,45,35,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242544.333333,c


In [22]:
#prb.to_csv('predicties TK alle data, zonder wox.csv', encoding='utf-16', index=False)